# Preprocessing

설명 주저리주저리

In [ ]:
import pandas as pd
import numpy as np

from itertools import chain

## Load Data

In [ ]:
data = pd.read_csv('New_Data/TCGA_GTEX_SLC.csv')

## 1) Remain sample 01

같은 사람이 여러 개의 데이터 가지고 있어 sample 번호 01로 끝나는 row만 남기고 제거

In [ ]:
data_t = data[data['sample'].str.contains('TCGA')] # TCGA만 고르기
data_t_01 = data_t[data_t['sample'].str.endswith('-01')] # 01만 남기기
data_t_01.reset_index(drop=True, inplace=True) 

In [ ]:
data_g = data[data['sample'].str.contains('GTEX')] # GTEX만 고르기

In [ ]:
data_all = pd.concat([data_t_01, data_g]) # 데이터 합치기
data_all.reset_index(drop=True, inplace=True)

## 2) GTEX-TCGA Matching



In [ ]:
df = data_all.copy()

In [ ]:
# TCGA, GTEx 리스트 생성
TCGA_list = df.loc[df['TCGA_GTEX_main_category'].str.contains('TCGA'), 'TCGA_GTEX_main_category'].unique()
GTEX_list = df.loc[df['TCGA_GTEX_main_category'].str.contains('GTEX'), 'TCGA_GTEX_main_category'].unique()

# GTEX - TCGA 매칭 리스트 생성
matching_list = [['GTEX Adrenal Gland', 'TCGA Adrenocortical Cancer'],
                  ['GTEX Bladder', 'TCGA Bladder Urothelial Carcinoma'],
                  ['GTEX Breast', 'TCGA Breast Invasive Carcinoma'],
                  ['GTEX Cervix Uteri', 'TCGA Cervical & Endocervical Cancer'],
                  ['GTEX Colon', 'TCGA Colon Adenocarcinoma', 'TCGA Rectum Adenocarcinoma'],
                  ['GTEX Blood', 'TCGA Diffuse Large B-Cell Lymphoma', 'TCGA Thymoma'],
                  ['GTEX Esophagus', 'TCGA Esophageal Carcinoma'],
                  ['GTEX Brain', 'TCGA Glioblastoma Multiforme', 'TCGA Brain Lower Grade Glioma'],
                  ['GTEX Kidney', 'TCGA Kidney Chromophobe', 'TCGA Kidney Clear Cell Carcinoma', 'TCGA Kidney Papillary Cell Carcinoma'],
                  ['GTEX Liver', 'TCGA Liver Hepatocellular Carcinoma'],
                  ['GTEX Lung', 'TCGA Lung Adenocarcinoma', 'TCGA Lung Squamous Cell Carcinoma'],
                  ['GTEX Ovary', 'TCGA Ovarian Serous Cystadenocarcinoma'],
                  ['GTEX Pancreas', 'TCGA Pancreatic Adenocarcinoma'],
                  ['GTEX Prostate', 'TCGA Prostate Adenocarcinoma'],
                  ['GTEX Skin', 'TCGA Skin Cutaneous Melanoma'],
                  ['GTEX Stomach', 'TCGA Stomach Adenocarcinoma'],
                  ['GTEX Testis', 'TCGA Testicular Germ Cell Tumor'],
                  ['GTEX Thyroid', 'TCGA Thyroid Carcinoma'],
                  ['GTEX Uterus', 'TCGA Uterine Carcinosarcoma', 'TCGA Uterine Corpus Endometrioid Carcinoma']]

In [ ]:
# npy 파일 내보내기
np.save('New_Data/mathing_list.npy', np.array(matching_list), allow_pickle=True)

* csv 파일 생성
    * SLC column 오름차순 정렬
    * Matching list에 포함된 row만 남기고 제거

In [ ]:
# SLC column list
slc = df.columns.values[1:-1]

# SLC column 오름차순으로 sort
col_sorted = ['sample'] + np.sort(slc).tolist() + ['TCGA_GTEX_main_category']

# 필요한 column만 indexing
df = df[col_sorted]

In [ ]:
# list flatten: 2D -> 1D
matching_flatten = list(chain.from_iterable(matching_list))

In [ ]:
# list에 포함된 row만 남기고 제거
df_ = df[df['TCGA_GTEX_main_category'].isin(matching_flatten)]
df_.reset_index(drop=True, inplace=True)

In [ ]:
# GTEX & TCGA 개수 출력
matched_TCGA = [t[1:] for t in matching_list] # TCGA list
for i in range(len(matching_list)):
    len_gtex = df_[df_['TCGA_GTEX_main_category']==matching_list[i][0]].shape[0] # GTEX 개수
    len_tcga = 0
    for m in matched_TCGA[i]: # GTEX와 매칭된 TCGA
        len_tcga += df_[df_['TCGA_GTEX_main_category']==m].shape[0] # TCGA 개수
    # 출력
    print(matching_list[i][0], ':', len_gtex, ' / ', matched_TCGA[i], ':', len_tcga)

## 3) Category Labeling

In [ ]:
data = matching_list # GTEX&TCGA matching list
data1 = df_.copy() # SLC 발현량 table

In [ ]:
data1['GTEx']='GTEX' #SLC 발현량 table에 GTEx를 기준으로 labeling

for k in range(len(data1)):
    for i in range(len(data)):
        for j in range(0,len(data[i])):
            if (data1['TCGA_GTEX_main_category'][k]==data[i][j]):
                data1['GTEx'][k]=data[i][0]

for i in range(len(data1)): #GTEx list에서 장기이름만 남기기
    data1['GTEx'][i]=data1['GTEx'][i].split(' ')[1]

data1.rename(columns={'GTEx':'cancer'},inplace=True) #GTEx 명칭을 cancer로 변경
data1['cancer_label']=data1['cancer'] #cancer_label 명칭을 cancer로 변경

for i in range(len(data)):
    for j in data[i]:
        for k in range(len(data1)):
            if data1['TCGA_GTEX_main_category'][k] == j:
                data1['cancer_label'][k] = i

data1['cancer_label_GTEx'] = data1['cancer_label']
data1.loc[data1['TCGA_GTEX_main_category'].str.contains('GTEX'),'cancer_label_GTEx'] = 100 #GTEx label을 100으로 둔 column 추가

data1.rename(columns={'cancer_label':'label','cancer_label_GTEx':'label_GTEx_100'},inplace=True)

In [ ]:
# 최종 table 내보내기
data1.to_csv('New_Data/TCGA_GTEX_matched_label.csv',index=False)